# IMAGE AUGMENTATION:
- a technique that can be used to artificially expand the size of a training set by creating modified data from the existing one.
- applyies different *transformations to original images* which results in multiple transformed copies of the same image. 
- Each copy, however, is different from the other in certain aspects depending on the augmentation techniques we apply like shifting, rotating, flipping, etc.
- used to expand the size of our dataset + incorporate a level of variation in the dataset : allows the model to generalize better on unseen data. 
- USE *Keras ImageDataGenerator*
- Used to prevent overfitting & when the sample size is very small.
- Also helps to improve the performance of the model.

## Data augmentation in TensorFlow and Keras: To augment images when using TensorFlow or Keras as our DL framework, we can:

1. Write our own augmentation pipelines or layers using tf.image.
2. Use Keras preprocessing layers
3. Use ImageDataGenerator

# Setup the data generators : Setting up data generators in Keras using ImageDataGenerator allows efficient loading and preprocessing of image data for training and validation. 


### Keras ImageDataGenerator class :
- generates batches of tensor images with real-time DA while your model is still training. ------ [*real-time data augmentation*]
- You can apply any random transformation on each training image as it is passed to the model.
- saves memory + model becomes robust
- Creates a large corpus of similar images without having to worry about collecting new images, which is not feasible in a real-world scenario.

- It ensures : model receives new variations of the images at each epoch. But it only returns the transformed images and does not add it to the original corpus of images.[seeing original images multiple times : *Overfiting*]
- requires lower memory usage :
   * Without using this class : we load all the images at once
   * Using it : we load the images in batches which saves a lot of memory

* The ImageDataGenerator() class has 3 methods :
- flow(), flow_from_directory() and flow_from_dataframe() to read the images from a big numpy array and folders containing images.
* flow_from_directory : allows you to read the images directly from the directory and augment them while the neural network model is learning on the training data.
- The directory must be set to the path where your ‘n’ classes of folders are present.
- The target_size : size of your input images, every image will be resized to this size.
- color_mode: if the image is either black and white or grayscale set “grayscale” or if the image has three color channels, set “rgb”.
- batch_size: No. of images to be yielded from the generator per batch.
- class_mode: Set “binary” if you have only two classes to predict, if not set to“categorical”, in case if you’re developing an Autoencoder system, both    input and the output would probably be the same image, for this case set to “input”.
- shuffle: Set True if you want to shuffle the order of the image that is being yielded, else set False.
- seed: Random seed for applying random image augmentation and shuffling the order of the image.

### Random Rotations:
- Rotate images :[0 and 360 degrees] -- providing an integer value in the *rotation_range* argument.
- So when image is rotated : some pixels will be moved outside the image & leave an empty space that needs to be filled in.
- We can fill this in different ways -- like constant value or nearest pixel values, etc. 
- This is specified in the *fill_mode* argument and the default value is *nearest : simply replaces the empty area with the nearest pixel values*

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Size of the image: 48x48 pixels
pic_size = 48                                                       #Specifies the size of each image (48x48 pixels in this case)


# Input path for the images
base_path = "/Users/pratiksha/Documents/Pratiksha/GitHub/Face-expression-recognition-with-Deep-Learning/images"

# number of images to feed into the NN for every batch
batch_size = 16     #Specifies the number of images to feed into the neural network for every batch during training and validation.

train_datagen = ImageDataGenerator(
    rescale=1.0/255,              # Normalize pixel values to [0, 1]
    rotation_range=20,            # Randomly rotate images in the range (degrees)
    width_shift_range=0.2,        # Randomly translate images horizontally
    height_shift_range=0.2,       # Randomly translate images vertically
    shear_range=0.2,              # Apply shearing transformations
    zoom_range=0.2,               # Randomly zoom images
    horizontal_flip=True,         # Randomly flip images horizontally
    fill_mode='nearest'           # Filling method for points outside boundaries
)

                            #ImageDataGenerator : it is used to generate a batch of images with some random transformations
validation_datagen = ImageDataGenerator(rescale=1.0/255)

In [14]:
#flow_from_directory() : allows to read the images directly from the directory & augment them while the NN model is learning on the training data.

# Creates instances of ImageDataGenerator for training (datagen_train) and validation (datagen_validation). 
# These generators will handle data augmentation, scaling, and other preprocessing tasks.

train_generator = train_datagen.flow_from_directory(base_path + "/train",            # flow_from_directory: Generates batches of augmented/normalized data from image files in the train directory.
                                                    target_size=(pic_size,pic_size), # Resizes images to (pic_size, pic_size) pixels.
                                                    color_mode="grayscale",           # Converts images to grayscale format.
                                                    batch_size=batch_size,            # Number of images per batch to be yielded from the generator.
                                                    class_mode='categorical',         # Returns one-hot encoded labels for multi-class classification.
                                                    shuffle=True)                     #  Shuffles the order of images after every epoch.




#class_mode: 1.categorical: for multi-class classification problems This means that the target output will be a binary matrix representation of the classes.
#            2.binary: for binary classification problems where the labels are 0 or 1. 
#            3.sparse: for multi-class classification problems where the labels are integers.useful when the number of classes is large.
#            4.input: for autoencoders. It returns the input unchanged.
#            5.none: if you don't want any labels returned.


Found 28821 images belonging to 7 classes.


- In above and below code we did: base + "train" and base + "validation" because: our your directory structure might look something like this:
/dataset
    /train
        /class1
        /class2
        ...
    /validation
        /class1
        /class2
        ...

* base_path: This is the root path pointing to the parent directory of train
* train": This specifies the subdirectory within base_path that contains the training images.

In [15]:
validation_generator = validation_datagen.flow_from_directory(base_path + "/validation", # fetches batches of validation data from the validation directory.
                                                    target_size=(pic_size,pic_size),
                                                    color_mode="grayscale",
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=False)
# Training Data:   Use shuffle = True to ensure the data is randomized and to help the model generalize better.
# Validation Data: Use shuffle = False to maintain a consistent evaluation set and ensure that the validation metrics are reliable.

Found 7066 images belonging to 7 classes.


In [16]:
# Compute class weights for the training data
classes = train_generator.classes  # Extract class indices from the train generator
class_weights = compute_class_weight('balanced', classes=np.unique(classes), y=classes)
class_weights_dict = dict(enumerate(class_weights))
print("Computed class weights:", class_weights_dict)


Computed class weights: {0: 1.031125898894494, 1: 9.443315858453474, 2: 1.0034817729187702, 3: 0.5747188322565207, 4: 0.8264322991340254, 5: 0.8337962159347335, 6: 1.2846445286382884}


NOTE :
There are various image augmentation techniques used:
1. Geometric transformations : rotation,randomly flip,crop or translate images.
2. Color space transformations : change RGB color channels and intensity of any color
3. Kernel filters : sharpen or blur an image
4. Random erasing: delete a part of the initial image
5. Mixing images 

- Deep learning models are trained by being fed with batches of data. Keras has a very useful class to automatically feed data from a directory: ImageDataGenerator.

- It can also perform data augmentation while getting the images (randomly rotating the image, zooming, etc.). This method is often used as a way to artificially get more data when the dataset has a small size.

- The function flow_from_directory() specifies how the generator should import the images (path, image size, colors, etc.).

* Found 28821 images belonging to 7 classes: indicates that in the training dataset (base_path + "train"), there are a total of 28,821 images distributed among 7 classes.
* Found 7066 images belonging to 7 classes: Similarly, in your validation dataset (base_path + "validation"), there are 7,066 images also distributed among the same 7 classes.


1. Initial Image Checking: Earlier, we used a loop to display a subset of images (specifically, the first 5 images) from each expression category (angry, disgust, fear, happy, neutral, sad, surprise) in the training data. This was likely done to visually inspect the data and ensure that images are loaded correctly and represent different facial expressions.

2. Setting up Data Generators: After verifying the data, we proceeded to set up 'ImageDataGenerator' instances for both training and validation datasets. These generators will be used to feed batches of images into your neural network during the training process.

3. Batch Size: We specified a batch_size of 128 when setting up the train_generator and validation_generator. This means that during training, the neural network will process 128 images at a time (or as many as can fit into memory for the hardware configuration). This batch size is a common parameter in deep learning training and affects how many images are processed before updating the model's weights.

## QUESTIONS:
1. USE of 'train_generator' & 'validation_generator' : 
- Instances of ImageDataGenerator from Keras that handle data preprocessing and augmentation (if specified). 
- They generate batches of images and their corresponding labels directly from directories of images, allowing you to work with large datasets that don't fit into memory.
- 'train_generator' : used during the training phase of your neural network. It provides batches of images to the model, allowing it to update its weights based on the gradients computed from these batches.
- validation_generator : used to evaluate the model's performance on a separate set of data that the model hasn't seen during training. It helps monitor the model's generalization ability and prevents overfitting.

2. BATCH SIZE:
- Batch size refers to the number of samples (images, in this case) that the model processes at a time before updating the weights.
When you set batch_size=128, the model will process 128 images in each iteration (batch) during training.
Benefits:

* Efficiency: Processing data in batches is more memory efficient. Instead of loading all images into memory at once, which may not be feasible for large datasets, batches allow you to work with manageable chunks.
* Gradient Descent: Batch processing allows for more stable gradient descent updates. It computes gradients based on the average loss across the batch, providing smoother convergence towards the optimal weights.

*References*
- https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
- https://vijayabhaskar96.medium.com/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720
- https://www.analyticsvidhya.com/blog/2020/08/image-augmentation-on-the-fly-using-keras-imagedatagenerator/
- https://neptune.ai/blog/data-augmentation-in-python#:~:text=Data%20augmentation%20is%20a%20technique,data%20from%20the%20existing%20one.